In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import os
import pickle
import numpy as np

CIFAR_DIR = "./cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

W0901 18:04:36.525273 140579887044416 deprecation.py:323] From /home/shungao/application/anaconda3/lib/python3.7/site-packages/tensorflow/python/compat/v2_compat.py:65: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


['data_batch_5', 'readme.html', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'test_batch', 'batches.meta']


In [2]:
# tensorboard
# 1. 指定面板图上显示的变量
# 2. 训练过程中将这些变量计算出来，输出到文件中
# 3. 文件解析 ./tensorboard --logdir=dir.

In [3]:
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        u = pickle._Unpickler(f)
        u.encoding = 'latin1'
        data = u.load()
        return data['data'], data['labels']

class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data
        self._labels = np.hstack(all_labels)
        
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self, batch_size):
        """return batch_size examples as a batch"""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples size")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, "data_batch_%d" % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, "test_batch")]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

# batch_data, batch_labels = train_data.next_batch(10)
# print(batch_data)
# print(batch_labels)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [4]:
batch_size = 20

# [None, 3072]
x = tf.placeholder(tf.float32, [batch_size, 3072])

x_image = tf.reshape(x, [-1, 3, 32, 32])

# [None]
y = tf.placeholder(tf.int64, [batch_size])

# is training -> boolean
is_training = tf.placeholder(tf.bool, [])

# 32 * 32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

x_image_arr = tf.split(x_image, num_or_size_splits=batch_size, axis=0)
result_x_image_arr = []

for x_single_image in x_image_arr:
    # x_single_image: [1, 32, 32, 3] -> [32, 32, 3]
    x_single_image = tf.reshape(x_single_image, [32, 32, 3])
    data_aug_1 = tf.image.random_flip_left_right(x_single_image)
    data_aug_2 = tf.image.random_brightness(data_aug_1, max_delta=63)
    data_aug_3 = tf.image.random_contrast(data_aug_2, lower=0.2, upper=1.8)
    x_single_image = tf.reshape(data_aug_3, [1, 32, 32, 3])
    result_x_image_arr.append(x_single_image)
result_x_image = tf.concat(result_x_image_arr, axis=0)
normal_result_x_image = result_x_image / 127.5 - 1

'''
def conv_wrapper(inputs,
                 name,
                 is_training,
                 output_channel=32,
                 kernel_size=(3, 3),
                 activation=tf.nn.relu,
                 padding='same'):
    """wrapper of tf.layers.conv2d"""
    # without bn: conv -> activation
    # with batch normalization: conv -> bn -> activation
    return tf.layers.conv2d(
        inputs,
        output_channel,
        kernel_size,
        padding = padding,
        activation = activation,
        name = name)

'''
def conv_wrapper(inputs,
                 name,
                 is_training,
                 output_channel = 32,
                 kernel_size = (3, 3),
                 activation = tf.nn.relu,
                 padding = 'same'):
    """wrapper of tf.layers.conv2d"""
    # without bn: conv -> activation
    # with batch normalization: conv -> bn -> activation
    with tf.name_scope(name):
        conv2d = tf.layers.conv2d(
            inputs,
            output_channel,
            kernel_size,
            padding = padding,
            activation = None,
            name = name + '/conv2d')
        bn = tf.layers.batch_normalization(
            conv2d,
            training = is_training,
            name = name + '/bn')
        return activation(bn, name = name + '/act')
# '''

def pooling_wrapper(inputs, name):
    """wrapper of tf.layers.max_pooling2d"""
    return tf.layers.max_pooling2d(
        inputs,
        (2, 2),
        (2, 2),
        name = name)

conv1_1 = conv_wrapper(normal_result_x_image, 'conv1_1', is_training)
conv1_2 = conv_wrapper(conv1_1, 'conv1_2', is_training)
conv1_3 = conv_wrapper(conv1_2, 'conv1_3', is_training)
pooling1 = pooling_wrapper(conv1_3, 'pool1')

conv2_1 = conv_wrapper(pooling1, 'conv2_1', is_training)
conv2_2 = conv_wrapper(conv2_1, 'conv2_2', is_training)
conv2_3 = conv_wrapper(conv2_2, 'conv2_3', is_training)
pooling2 = pooling_wrapper(conv2_3, 'pool2')

conv3_1 = conv_wrapper(pooling2, 'conv3_1', is_training)
conv3_2 = conv_wrapper(conv3_1, 'conv3_2', is_training)
conv3_3 = conv_wrapper(conv3_2, 'conv3_3', is_training)
pooling3 = pooling_wrapper(conv3_3, 'pool3')

flatten = tf.layers.flatten(pooling3)

y_ = tf.layers.dense(flatten, 10)


# 交叉熵损失
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> softmax
# y -> one_hot
# loss = ylogy_


# indices
predict = tf.argmax(y_, 1)
# [1, 0, 1, 1, 0, 1, 0 ...]
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

W0901 18:04:37.695171 140579887044416 deprecation.py:323] From <ipython-input-4-7952e03adda5>:68: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0901 18:04:37.985094 140579887044416 deprecation.py:323] From <ipython-input-4-7952e03adda5>:72: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
W0901 18:04:38.215128 140579887044416 deprecation.py:323] From <ipython-input-4-7952e03adda5>:82: max_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
W0901 18:04:

In [5]:
def variable_summary(var, name):
    """Constructs summary for statistics of a variable"""
    with tf.name_scope(name):
        mean = tf.reduce_mean(var)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('mean', mean)
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.histogram('histogram', var)

with tf.name_scope('summary'):
    variable_summary(conv1_1, 'conv1_1')
    variable_summary(conv1_2, 'conv1_2')
    variable_summary(conv1_3, 'conv1_3')
    variable_summary(conv2_1, 'conv2_1')
    variable_summary(conv2_2, 'conv2_2')
    variable_summary(conv2_3, 'conv2_3')
    variable_summary(conv3_1, 'conv3_1')
    variable_summary(conv3_2, 'conv3_2')
    variable_summary(conv3_3, 'conv3_3')
        
loss_summary = tf.summary.scalar('loss', loss)

accuracy_summary = tf.summary.scalar('accuracy', accuracy)

source_image = normal_result_x_image
inputs_summary = tf.summary.image('input_images', source_image)

merged_summary = tf.summary.merge_all()
merged_summary_test = tf.summary.merge([loss_summary, accuracy_summary])

LOG_DIR = './tensorboard'
run_label = 'run_vgg_tensorboard'
run_dir = os.path.join(LOG_DIR, run_label)
if not os.path.exists(run_dir):
    os.mkdir(run_dir)
train_log_dir = os.path.join(run_dir, 'train')
test_log_dir = os.path.join(run_dir, 'test')
if not os.path.exists(train_log_dir):
    os.mkdir(train_log_dir)
if not os.path.exists(test_log_dir):
    os.mkdir(test_log_dir)

In [ ]:
init = tf.global_variables_initializer()
#batch_size = 20
train_steps = 10000
test_steps = 100
output_summary_every_steps = 1000

with tf.Session() as sess:
    sess.run(init)
    train_writer = tf.summary.FileWriter(train_log_dir, sess.graph)
    test_writer = tf.summary.FileWriter(test_log_dir)
    
    fixed_test_batch_data, fixed_test_batch_labels = test_data.next_batch(batch_size)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        eval_ops = [loss, accuracy, train_op]
        should_outputs_summary = ((i + 1) % output_summary_every_steps == 0)
        if should_outputs_summary:
            eval_ops.append(merged_summary)
        eval_ops_results = sess.run(
            eval_ops,
            feed_dict={
                x: batch_data,
                y: batch_labels,
                is_training: True})
        loss_val, acc_val = eval_ops_results[0:2]
        if should_outputs_summary:
            train_summary_str = eval_ops_results[-1]
            train_writer.add_summary(train_summary_str, i + 1)
            test_summary_str = sess.run(
                [merged_summary_test],
                feed_dict = {
                    x: fixed_test_batch_data,
                    y: fixed_test_batch_labels,
                    is_training: False})[0]
            test_writer.add_summary(test_summary_str, i + 1)
        if (i + 1) % 100 == 0:
            print('[Train] step: %d, loss: %4.5f, acc: %4.5f' % (i + 1, loss_val, acc_val))
        if (i + 1) % 500 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels,
                        is_training: False})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] step: %d, acc: %4.5f' % (i + 1, test_acc))

[Train] step: 100, loss: 1.99838, acc: 0.25000
[Train] step: 200, loss: 1.89129, acc: 0.20000
[Train] step: 300, loss: 1.82212, acc: 0.35000
[Train] step: 400, loss: 1.83827, acc: 0.10000
[Train] step: 500, loss: 1.53793, acc: 0.55000
(10000, 3072)
(10000,)
[Test ] step: 500, acc: 0.18000
[Train] step: 600, loss: 1.95192, acc: 0.40000
[Train] step: 700, loss: 1.66699, acc: 0.25000
[Train] step: 800, loss: 1.36280, acc: 0.50000
[Train] step: 900, loss: 1.67972, acc: 0.55000
[Train] step: 1000, loss: 1.80662, acc: 0.30000
(10000, 3072)
(10000,)
[Test ] step: 1000, acc: 0.23300
[Train] step: 1100, loss: 1.52778, acc: 0.50000
[Train] step: 1200, loss: 1.10392, acc: 0.70000
[Train] step: 1300, loss: 1.39490, acc: 0.55000
[Train] step: 1400, loss: 1.26297, acc: 0.50000
[Train] step: 1500, loss: 1.48139, acc: 0.50000
(10000, 3072)
(10000,)
[Test ] step: 1500, acc: 0.24800
[Train] step: 1600, loss: 0.96903, acc: 0.65000
[Train] step: 1700, loss: 1.46154, acc: 0.50000
[Train] step: 1800, loss: 